# Setup #

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


'andes03.cp.lsst.org'

In [3]:
import os
import sys
import asyncio
import logging

import pandas as pd

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

Update leap second table
current_tai uses the system TAI clock


In [4]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [5]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [6]:
domain = salobj.Domain()

In [7]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

mtmount: Adding all resources.
mtptg: Adding all resources.
mtaos: Adding all resources.
mtm1m3: Adding all resources.
mtm2: Adding all resources.
mthexapod_1: Adding all resources.
mthexapod_2: Adding all resources.
mtrotator: Adding all resources.
mtdome: Adding all resources.
mtdometrajectory: Adding all resources.


In [8]:
await mtcs.start_task

zenithAngle DDS read queue is full (100 elements); data may be lost
temperature DDS read queue is full (100 elements); data may be lost
tangentForce DDS read queue is full (100 elements); data may be lost
tangentEncoderPositions DDS read queue is full (100 elements); data may be lost
tangentActuatorSteps DDS read queue is full (100 elements); data may be lost
powerStatus DDS read queue is full (100 elements); data may be lost
positionIMS DDS read queue is full (100 elements); data may be lost
position DDS read queue is full (100 elements); data may be lost
netMomentsTotal DDS read queue is full (100 elements); data may be lost
Read historical data in 0.17 sec
netForcesTotal DDS read queue is full (100 elements); data may be lost
ilcData DDS read queue is full (100 elements); data may be lost
Read historical data in 0.20 sec
forceBalance DDS read queue is full (100 elements); data may be lost
displacementSensors DDS read queue is full (100 elements); data may be lost
axialForce DDS read

[None, None, None, None, None, None, None, None, None, None]

electrical DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is filling: 86 of 100 elements
powerSupplyData DDS read queue is full (100 elements); data may be lost
timeAndDate DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is full (100 elements); data may be lost


In [9]:
comcam = ComCam(domain=domain, log=log)

azimuth DDS read queue is full (100 elements); data may be lost
application DDS read queue is full (100 elements); data may be lost
application DDS read queue is filling: 88 of 100 elements
pidData DDS read queue is full (100 elements); data may be lost
mountStatus DDS read queue is full (100 elements); data may be lost
ccwFollowingError DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is filling: 89 of 100 elements
inclinometerData DDS read queue is full (100 elements); data may be lost
mountPosition DDS read queue is full (100 elements); data may be lost
currentTargetStatus DDS read queue is full (100 elements); data may be lost
imsData DDS read queue is full (100 elements); data may be lost
hardpointMonitorData DDS read queue is full (100 elements); data may be lost
hardpointActuatorData DDS read queue is full (100 elements); data may be lost
forceActuatorData DDS read queue is full (10

In [10]:
comcam.set_rem_loglevel(40)

In [11]:
await comcam.start_task

[None, None, None]

In [ ]:
await comcam.enable()

# Here we perform a test to slew and take an image #

In [ ]:
target = await mtcs.find_target(el=80, az=120, mag_limit=8)
print(target)

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-1.5)

Recover pointing fault below, target movement forced rotator to track into difference limit of -2.0 deg. Will need to select a new target with a better tracking vector

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

A lower elevation will mean the rotator will move slower to keep up, and an azimuth of 120 E of N means the target is in the Southeast of the sky, rising. When rising, we will need to have a positive rot offset applied to start as the rotator will advance in the negative direction. We will need the cell below to avoid sending two targets at the same time.

rotator/hex went to fault because we did not fully clear the previous target we sent, we need to clear the faults on both. This can be done from the engineering screens or using the command below

In [ ]:
#use this to clear the fault status
#await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [ ]:
await mtcs.rem.mtrotator.cmd_stop.start(timeout=5)

In [12]:
target = await mtcs.find_target(el=60, az=120, mag_limit=8)
print(target)

HD 119154


In [13]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

Slewing to HD 119154: 13 42 17.5515 -40 26 27.948
Setting rotator physical position to 1.9 deg. Rotator will track sky.
Camera cable wrap following disabled in MTMount.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
Mount target: private_revCode: 261ad639, private_sndStamp: 1632412508.400535, private_rcvStamp: 1632412508.401246, private_seqNum: 918, private_identity: MTMount, private_origin: 103063, private_host: 0, elevation: 59.69083674368315, elevationVelocity: 0.003137983574026249, azimuth

telescope is in position, let's take a comcam image!

In [14]:
await comcam.take_object(15)

Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001


array([2021092300001])

In [15]:
await mtcs.stop_tracking()

Stop tracking.


The image landed correctly and even has some correct info in the file header, later we will need to do a more careful analysis of the header information

# Now we are moving to on mtaos testing #

initialize an array of wavefront error values to be passed directly to the mtaos

In [17]:
import numpy as np
wavefront_errors = np.zeros(19)

In [19]:
wavefront_errors[1] += 0.1 # add 0.1 um of oblique astigmatism

In [20]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

This command primes the corrections, the issueCorrection command is needed to actually command them to be sent

In [21]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [23]:
await mtcs.rem.mtaos.cmd_resetCorrection.start()

In [26]:
wavefront_errors[1] = 0
wavefront_errors[3] = 1.0 # add 1.0 um of z7

In [25]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

actuators DDS read queue is filling: 10 of 100 elements


electrical DDS read queue is filling: 13 of 100 elements
application DDS read queue is filling: 13 of 100 elements


In [27]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [28]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [30]:
wavefront_errors[3] = 1.0 # add 1.0 um of z7

In [31]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [32]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

# Moving to a failure test of how the mtaos handles a rejected command #
JIRA LVV-T2193

In [33]:
await mtcs.rem.mtaos.cmd_resetCorrection.start()

In [34]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [35]:
wavefront_errors

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [36]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [37]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [38]:
await mtcs.set_state(salobj.State.DISABLED, components=["mthexapod_2"])

[mthexapod_2]::[<State.ENABLED: 2>, <State.DISABLED: 1>]
All components in <State.DISABLED: 1>.


In [39]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [40]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=763213325, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: Failed to apply correction to: ['m2hex']. ")

In [41]:
await mtcs.set_state(salobj.State.ENABLED, components=["mthexapod_2"])

[mthexapod_2]::[<State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [42]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [43]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [44]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [45]:
await mtcs.rem.mtaos.cmd_rejectCorrection.start()

In [46]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [47]:
wavefront_errors[3] += 1.0

In [48]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [49]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

[mtaos]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [50]:
await mtcs.lower_m1m3()

M1M3 current detailed state {<DetailedState.ACTIVEENGINEERING: 11>, <DetailedState.ACTIVE: 7>}, executing command...
process as completed...
M1M3 detailed state 8
mtm1m3: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
M1M3 detailed state 5


In [52]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

[mtm1m3]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [54]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

[mtm2]::[<State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [55]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

[mthexapod_1]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [56]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

[mthexapod_2]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [57]:
await mtcs.standby()

[mtmount]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtptg]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtaos]::[<State.STANDBY: 5>]
[mtm1m3]::[<State.STANDBY: 5>]
[mtm2]::[<State.STANDBY: 5>]
[mthexapod_1]::[<State.STANDBY: 5>]
[mthexapod_2]::[<State.STANDBY: 5>]
[mtrotator]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtdome]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtdometrajectory]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [58]:
await comcam.standby()

[cccamera]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[ccheaderservice]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[ccarchiver]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


# Appendix #

In [ ]:
# Setting MTMtg to STANDBY so Tiago can change the slew limits which should allows
#  us moving to rot=-1.5
await mtcs.set_state(salobj.State.STANDBY, components=["mtptg"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-1.5)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

In [ ]:
await mtcs.rem.mthexapod_1.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

In [ ]:
target = await mtcs.find_target(el=60, az=120, mag_limit=8)
print(target)

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky)

In [ ]:
# await mtcs.stop_tracking()

In [ ]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

In [ ]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

In [ ]:
await comcam.take_object(1)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.Physical, rot=0)

In [ ]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.Physical, rot=0)

In [ ]:
await comcam.get_available_filters()

In [ ]:
await comcam.setup_filter('r_03')

In [ ]:
await mtcs.rem.mthexapod_1.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

In [ ]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

In [ ]:
await comcam.take_object(15)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await comcam.standby()